In [1]:
import pandas as pd
import numpy as np
import time
import urllib.request
import os
import pickle
from os import path

In [ ]:
with open("./멀티캠퍼스DF.pkl","br") as f:
    mcdf = pickle.load(f)

In [ ]:
mcdf['일자']=pd.to_numeric(mcdf['일자'])

In [ ]:
mcdf=mcdf.loc[mcdf['일자']>20211230]
mcdf=mcdf.loc[mcdf['일자']<20230101]
mcdf

In [ ]:
mcdf=mcdf.sort_values(by=['일자'])
mcdf

In [ ]:
mcdf['글자수']=mcdf['본문'].str.count('')-1
mcdf

In [ ]:
mcdf=mcdf.loc[mcdf['글자수']>2]
mcdf

In [ ]:
mcdf=mcdf.loc[mcdf['글자수']<1000]
mcdf

In [ ]:
newsdf=(mcdf.loc[(mcdf['일자']==20220103)])[['본문']]
newsdf

In [ ]:
newslist=newsdf['본문'].values.tolist()
newslist

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-FinBert-SC')
model=AutoModelForSequenceClassification.from_pretrained('snunlp/KR-FinBert-SC')

senti_classifier = pipeline(task='text-classification',model=model,tokenizer=tokenizer)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.0.1+cpu)
    Python  3.9.13 (you have 3.9.0)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [3]:
def allsenticlassifier(allnewsdf,day):
    newsdf=(allnewsdf.loc[(allnewsdf['일자']==day)])[['본문']]
    newslist=newsdf['본문'].values.tolist()
    x = newslist
    y = [x['label'] for x in senti_classifier(x)]
    
    neutralC = y.count('neutral')
    negativeC = y.count('negative')
    positiveC = y.count('positive')
    
    addC=neutralC+negativeC+positiveC
    
    neutralP=neutralC/addC
    negativeP=negativeC/addC
    positiveP=positiveC/addC
    
    return neutralP,negativeP,positiveP

In [ ]:
neutralPs=[]
negativePs=[]
positivePs=[]

In [ ]:
mcdf_day_listint

In [ ]:
mcdf_day_list=mcdf['일자'].to_list()
mcdf_day_setint=set(map(int,mcdf_day_list))
mcdf_day_listint = list(mcdf_day_setint)
mcdf_day_listint.sort()

In [ ]:
for day in mcdf_day_listint:
    neutralP,negativeP,positiveP=allsenticlassifier(mcdf,day)
    neutralPs.append(neutralP)
    negativePs.append(negativeP)
    positivePs.append(positiveP)
    print(day)

In [ ]:
x = newslist
y = [x['label'] for x in senti_classifier(x)]

In [ ]:
len(mcdf_day_listint)

In [ ]:
#저장


with open("./mcneutralPs.pickle","bw") as f:
    pickle.dump(neutralPs, f)
    
with open("./mcnegativePs.pickle","bw") as f:
    pickle.dump(negativePs, f)

with open("./mcpositivePs.pickle","bw") as f:
    pickle.dump(positivePs, f)